# Westeros Tutorial Part X - Introducing combined heat and power (CHP)

In the first part of the Westeros tutorial, only a single secondary energy carrier - electricity - was produced to satisfy the service demand for lighting. Here we add a second demand for heating and introduce combined heat and power (CHP) to the coal power plant as an option to produce heat. 

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform(dbtype = 'HSQLDB')

In [ ]:
model = 'Westeros Electrified'

base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'chp','introducing a heat demand and combined heat and power (CHP)',
                  keep_solution=False)
scen.check_out()

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Adding heat demand

In [ ]:
# first we introduce a new commodity heat and introduce a demand for it
scen.add_set('commodity', 'heat')

# obtain model horizon from baseline model 
model_horizon = [700, 710, 720]

# generate heat demand time series at 10% of lighting demand (see Westeros baseline tutorial)
heat_profile = pd.Series([10, 15, 19],
                        index=pd.Index(model_horizon, name='Time'))

# create data frame for heat demand parameter 
heat_demand = pd.DataFrame({
        'node': country,
        'commodity': 'heat',
        'level': 'secondary',
        'year': model_horizon,
        'time': 'year',
        'value': (heat_profile).round(),
        'unit': 'GWa',
    })

# add new demand via add_par to a MESSAGEix
scen.add_par("demand", heat_demand)

In [ ]:
# add a second operating mode for the coal power plant that allows varying the ratio of electricity to heat produced
scen.add_set("mode", "max_heat")

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']

# Add output coefficients for electricity and hear to the second operating mode of the coal powerplant

base_output = {
    'node_loc': country,
    'node_dest': country,
    'technology': 'coal_ppl',
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
    'time_dest': 'year',
    'level': 'secondary', 
    'unit': '-',
}

# electric efficiency is reduced to about 32% (from about 40% in standard operation mode without heat geenration) with total efficiency going up to about 85%
coal_out_elc = make_df(base_output, commodity='electricity', value=0.8)
coal_out_heat = make_df(base_output, commodity='heat', value=1.3)

scen.add_par('output', coal_out_elc)
scen.add_par('output', coal_out_heat)

In [ ]:
# Adding CO2 emissions to the second operating mode of the coal powerplant
#emission_factor = {
#    'node_loc': country,
#    'technology': 'coal_ppl',
#    'year_vtg': vintage_years,
#    'year_act': act_years,
#    'mode': 'max_heat',
#    'unit': 'tCO2/kWa',
#    'value': 7.4,    
#}

# Adding variable costs (incl. fuel costs) to the second operating mode of the coal powerplant
var_cost = {
    'node_loc': country,
    'technology': 'coal_ppl',
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'max_heat',
    'time': 'year',
    'unit': 'USD/kWa',
    'value': 30,
}

#scen.add_par('emission_factor', emission_factor)
scen.add_par('var_cost', var_cost)

## Time to Solve the Model

In [ ]:
scen.commit(comment='introducing a heat demand and combined heat and power (CHP)')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

## Plotting Results

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700)

### Activity

How much energy is generated in each time period from the different potential sources?

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Capacity

How much capacity of each plant is installed in each period?

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Heat Price

And how much does the heat cost? These prices are in fact **shadow prices** taken from the **dual variables** of the model solution. They reflect the marginal cost of electricity generation (i.e., the additional cost of the system for supplying one more unit of electricity), which is in fact the marginal cost of the most expensive generator.  

Note the price drop when the most expensive technology is no longer in the system.

In [ ]:
p.plot_prices(subset=['heat'], baseyear=True)

## Close the connection to the database

In [ ]:
mp.close_db()